In [98]:
import pandas as pd
pd.set_option("display.max_rows", 1000)

In [73]:
pop_df = pd.read_csv('./Resources/population_by_zip_2010.csv')

In [74]:
pop_df.shape

(1622831, 6)

In [75]:
pop_df['zipcode'].nunique()

33119

In [67]:
pop_df['gender'].iat[36]

nan

In [68]:
pop_df[(pop_df['minimum_age'].isnull())]

,population,minimum_age,maximum_age,gender,zipcode,geo_id
32,2865,NaN,NaN,male,56164,8600000US56164
36,27669,NaN,NaN,NaN,91324,8600000US91324
52,443,NaN,NaN,male,97037,8600000US97037
55,1019,NaN,NaN,male,14604,8600000US14604
57,132,NaN,NaN,male,74534,8600000US74534
...,...,...,...,...,...,...
1622775,8270,NaN,NaN,female,50263,8600000US50263
1622786,2081,NaN,NaN,male,88415,8600000US88415
1622791,5062,NaN,NaN,NaN,75709,8600000US75709
1622810,383,NaN,NaN,male,41835,8600000US41835


In [69]:
pop_df.dropna(subset=['minimum_age'],inplace=True)

In [70]:
pop_df[(pop_df['gender']=='NaN')]

,population,minimum_age,maximum_age,gender,zipcode,geo_id


In [71]:
pop_df.shape

(1523474, 6)

## Population Dataframe for upload to Postgres

In [49]:
population_df = pop_df[['population', 'minimum_age', 'gender', 'zipcode']].copy(deep=True)

In [40]:
population_df.head()

,population,minimum_age,gender,zipcode
0,50,30.0,female,61747
1,5,85.0,male,64120
2,1389,30.0,male,95117
3,231,60.0,female,74074
4,56,0.0,female,58042


In [79]:
income_df = pd.read_csv('./Resources/kaggle_income.csv', encoding='latin-1')

In [81]:
income_df[income_df['Primary'] == 'place']['Zip_Code'].value_counts()

78584    10
78582     7
66749     4
95076     4
21502     4
         ..
60060     1
45723     1
53578     1
19096     1
77546     1
Name: Zip_Code, Length: 2863, dtype: int64

In [88]:
income_df[income_df['Primary'] == 'place']

,id,State_Code,State_Name,State_ab,County,City,Place,Type,Primary,Zip_Code,Area_Code,ALand,AWater,Lat,Lon,Mean,Median,Stdev,sum_w
0,1011000,1,Alabama,AL,Mobile County,Chickasaw,Chickasaw city,City,place,36611,251,10894952,909156,30.771450,-88.079697,38773,30506,33101,1638.260513
1,1011010,1,Alabama,AL,Barbour County,Louisville,Clio city,City,place,36048,334,26070325,23254,31.708516,-85.611039,37725,19528,43789,258.017685
2,1011020,1,Alabama,AL,Shelby County,Columbiana,Columbiana city,City,place,35051,205,44835274,261034,33.191452,-86.615618,54606,31930,57348,926.031000
3,1011030,1,Alabama,AL,Mobile County,Satsuma,Creola city,City,place,36572,251,36878729,2374530,30.874343,-88.009442,63919,52814,47707,378.114619
4,1011040,1,Alabama,AL,Mobile County,Dauphin Island,Dauphin Island,Town,place,36528,251,16204185,413605152,30.250913,-88.171268,77948,67225,54270,282.320328
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32167,7201797,72,Puerto Rico,PR,Dorado Municipio,Dorado,Río Lajas,Community,place,646,787,3909197,19327,18.396174,-66.266020,25082,15941,23491,657.060040
32168,7201807,72,Puerto Rico,PR,Aguadilla Municipio,Aguadilla,San Antonio,Community,place,605,787,1252042,1016,18.492693,-67.100851,14072,11406,12862,602.628444
32169,7201817,72,Puerto Rico,PR,Aguas Buenas Municipio,Aguas Buenas,Santa Clara,Community,place,703,787,802453,5544,18.209290,-66.133445,12262,8931,9665,264.309067
32170,7201827,72,Puerto Rico,PR,Toa Alta Municipio,Toa Alta,Toa Alta,Urban,place,953,787,1623333,22071,18.388555,-66.249779,29580,21630,25324,792.727022


## Income DataFrame for upload to Postgres

In [89]:
income_df = income_df[['State_Name','County','City','Place','Type','Zip_Code','Mean','Median','Stdev']]

In [91]:
income_df.head()

,State_Name,County,City,Place,Type,Zip_Code,Mean,Median,Stdev
0,Alabama,Mobile County,Chickasaw,Chickasaw city,City,36611,38773,30506,33101
1,Alabama,Barbour County,Louisville,Clio city,City,36048,37725,19528,43789
2,Alabama,Shelby County,Columbiana,Columbiana city,City,35051,54606,31930,57348
3,Alabama,Mobile County,Satsuma,Creola city,City,36572,63919,52814,47707
4,Alabama,Mobile County,Dauphin Island,Dauphin Island,Town,36528,77948,67225,54270


In [122]:
income_df[['Zip_Code', 'City']].sort_values('Zip_Code')
income_df.rename(columns={'Zip_Code':'zipcode'}, inplace=True)

In [123]:
income_df.head()

,State_Name,County,City,Place,Type,zipcode,Mean,Median,Stdev
0,Alabama,Mobile County,Chickasaw,Chickasaw city,City,36611,38773,30506,33101
1,Alabama,Barbour County,Louisville,Clio city,City,36048,37725,19528,43789
2,Alabama,Shelby County,Columbiana,Columbiana city,City,35051,54606,31930,57348
3,Alabama,Mobile County,Satsuma,Creola city,City,36572,63919,52814,47707
4,Alabama,Mobile County,Dauphin Island,Dauphin Island,Town,36528,77948,67225,54270


In [100]:
inpatient_df = pd.read_csv('./Resources/inpatientCharges.csv')

In [101]:
inpatient_df.head()

,DRG Definition,Provider Id,Provider Name,Provider Street Address,Provider City,Provider State,Provider Zip Code,Hospital Referral Region Description,Total Discharges,Average Covered Charges,Average Total Payments,Average Medicare Payments
0,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,10001,SOUTHEAST ALABAMA MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,AL - Dothan,91,$32963.07,$5777.24,$4763.73
1,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,10005,MARSHALL MEDICAL CENTER SOUTH,2505 U S HIGHWAY 431 NORTH,BOAZ,AL,35957,AL - Birmingham,14,$15131.85,$5787.57,$4976.71
2,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,10006,ELIZA COFFEE MEMORIAL HOSPITAL,205 MARENGO STREET,FLORENCE,AL,35631,AL - Birmingham,24,$37560.37,$5434.95,$4453.79
3,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,10011,ST VINCENT'S EAST,50 MEDICAL PARK EAST DRIVE,BIRMINGHAM,AL,35235,AL - Birmingham,25,$13998.28,$5417.56,$4129.16
4,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,10016,SHELBY BAPTIST MEDICAL CENTER,1000 FIRST STREET NORTH,ALABASTER,AL,35007,AL - Birmingham,18,$31633.27,$5658.33,$4851.44


## Provider Dataframe

In [103]:
provider_df = inpatient_df[['Provider Id','Provider Name','Provider City','Provider State','Provider Zip Code']].copy(deep=True)

In [125]:
provider_df['Provider Id'].nunique()
provider_df.drop_duplicates(inplace=True)
# provider_df.duplicated()
provider_df.shape
provider_df.rename(columns={'Provider Zip Code':'zipcode'}, inplace=True)
provider_df.head()

,Provider Id,Provider Name,Provider City,Provider State,zipcode
0,10001,SOUTHEAST ALABAMA MEDICAL CENTER,DOTHAN,AL,36301
1,10005,MARSHALL MEDICAL CENTER SOUTH,BOAZ,AL,35957
2,10006,ELIZA COFFEE MEMORIAL HOSPITAL,FLORENCE,AL,35631
3,10011,ST VINCENT'S EAST,BIRMINGHAM,AL,35235
4,10016,SHELBY BAPTIST MEDICAL CENTER,ALABASTER,AL,35007


## InpatientCharges Dataframe

In [126]:
inpatientcharges_df = inpatient_df[['DRG Definition','Provider Id',' Total Discharges ', ' Average Covered Charges ',' Average Total Payments ', 'Average Medicare Payments']]
inpatientcharges_df.head()

,DRG Definition,Provider Id,Total Discharges,Average Covered Charges,Average Total Payments,Average Medicare Payments
0,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,10001,91,$32963.07,$5777.24,$4763.73
1,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,10005,14,$15131.85,$5787.57,$4976.71
2,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,10006,24,$37560.37,$5434.95,$4453.79
3,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,10011,25,$13998.28,$5417.56,$4129.16
4,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,10016,18,$31633.27,$5658.33,$4851.44
